In [1]:
import os
from groq import Groq
from dotenv import load_dotenv

load_dotenv()
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

In [22]:
def extract_triplets(text):
    system_prompt = """
    You are an expert Knowledge Graph engineer. Your task is to extract relationships from text in the form of (subject, relation, object) triplets.
    
    GUIDELINES:
    1. Subjects and Objects should be concise nouns or entities (e.g., "Ragas", "LLM", "Evaluation Framework").
    2. Relations should be verbs or verb phrases representing the link (e.g., "is designed for", "measures", "improves").
    3. If a sentence is complex, break it into multiple simple triplets.
    4. Ensure the output is strictly valid JSON.
    
    EXAMPLE:
    Text: "Ragas uses LLMs to automate the evaluation of RAG pipelines."
    Output: {
      "triplets": [
        ["Ragas", "uses", "LLMs"],
        ["Ragas", "automates", "evaluation of RAG pipelines"]
      ]
    }
    """
    completion = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {
                "role": "system", 
                "content":system_prompt
            },
            {
                "role": "user", 
                "content": f"Extract entities from this text: {text}"
            }
        ],
        temperature=0, # it should be  deterministic for KG extraction
        response_format={"type": "json_object"} # to get valid JSON back
    )
    return completion.choices[0].message.content

# Example 
sample_text = "Patient shows signs of Bradycardia which is often associated with the HCN4 gene mutation."
print(extract_triplets(sample_text))

{
  "triplets": [
       ["Patient", "shows", "signs of Bradycardia"],
       ["Bradycardia", "is associated with", "HCN4 gene mutation"]
   ]
}


In [23]:
import os
import json
from dotenv import load_dotenv
from groq import Groq
from langchain_experimental.text_splitter import SemanticChunker
from langchain_huggingface import HuggingFaceEmbeddings
from neo4j import GraphDatabase

In [24]:
load_dotenv()
groq_client = Groq(api_key=os.getenv("GROQ_API_KEY"))
embed_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
def get_propositions(text):
    response = groq_client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {
                "role": "system", 
                "content": (
                    "Decompose the input text into standalone, atomic facts. "
                    "Return the result in JSON format with a single key called 'propositions' "
                    "which contains a list of strings. "
                    "Example: {'propositions': ['fact 1', 'fact 2']}" 
                )
            },
            {
                "role": "user", 
                "content": f"Decompose this text: {text}"
            }
        ],
        response_format={"type": "json_object"}
    )
    
    content = json.loads(response.choices[0].message.content)
    return content.get("propositions", [])

In [26]:
def semantic_grouping(propositions):
    # SemanticChunker expects a single string or list of docs
    chunker = SemanticChunker(embed_model, breakpoint_threshold_type="percentile")
    full_text = " ".join(propositions)
    # This splits the text at points where the "meaning" shifts
    chunks = chunker.create_documents([full_text])
    return [chunk.page_content for chunk in chunks]

In [27]:
def save_to_neo4j(triplets):
    uri = os.getenv("NEO4J_URI")
    user = os.getenv("NEO4J_USERNAME") 
    password = os.getenv("NEO4J_PASSWORD")

    driver = GraphDatabase.driver(uri, auth=(user, password))
    
    with driver.session() as session:
        for triplet in triplets:
            # it's a valid 3-item list or not
            if isinstance(triplet, list) and len(triplet) == 3:
                s, r, o = triplet
                
                # Convention
                rel_type = str(r).replace(" ", "_").upper()
                
                session.run("""
                    MERGE (a:Entity {name: $s})
                    MERGE (b:Entity {name: $o})
                    WITH a, b
                    CALL apoc.merge.relationship(a, $rel_type, {}, {}, b) YIELD rel
                    RETURN rel
                """, s=str(s), rel_type=rel_type, o=str(o))
            else:
                print(f"Skipping malformed triplet: {triplet}")
                
    driver.close()

In [28]:
from langchain_community.document_loaders import PyPDFLoader, TextLoader, Docx2txtLoader
def load_local_file(file_path):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"The file {file_path} was not found.")

    ext = os.path.splitext(file_path)[-1].lower()
    
    if ext == ".pdf":
        loader = PyPDFLoader(file_path)
    elif ext == ".txt":
        loader = TextLoader(file_path, encoding="utf-8")
    elif ext == ".docx":
        loader = Docx2txtLoader(file_path)
    else:
        raise ValueError(f"Unsupported file format: {ext}")

    docs = loader.load()
    return "\n\n".join([doc.page_content for doc in docs])

In [29]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
structural_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000, 
    chunk_overlap=200,
    separators=["\n\n", "\n", ".", " "]
)

In [30]:
from tqdm import tqdm

target_file = r"C:\Users\Cengizhan\Desktop\CMPE492-Project-Rag-Pipeline\Documents\Ragas\ragas_2309.15217v2.pdf"
print(f"Loading file: {target_file}")
raw_content = load_local_file(target_file)
rough_chunks = structural_splitter.split_text(raw_content)

all_propositions = []

for segment in tqdm(rough_chunks, desc="Extracting Facts"):
    props = get_propositions(segment) 
    all_propositions.extend(props)
    
semantic_chunks = semantic_grouping(all_propositions)


Loading file: C:\Users\Cengizhan\Desktop\CMPE492-Project-Rag-Pipeline\Documents\Ragas\ragas_2309.15217v2.pdf


Extracting Facts: 100%|██████████| 22/22 [01:56<00:00,  5.27s/it]


In [33]:
semantic_chunks

['Ragas is a framework for reference-free evaluation of Retrieval Augmented Generation pipelines. RAG systems are composed of a retrieval and an LLM based generation module. RAG systems provide LLMs with knowledge from a reference textual database. RAG systems reduce the risk of hallucinations by acting as a natural language layer between a user and textual databases. Evaluating RAG architectures is challenging because of several dimensions to consider. The dimensions to consider in RAG architectures include the ability to identify relevant and focused context passages. The dimensions to consider in RAG architectures include the ability to exploit passages in a faithful way. The dimensions to consider in RAG architectures include the quality of the generation itself. Ragas provides a suite of metrics to evaluate different dimensions without relying on ground truth human annotations. A framework like Ragas can contribute to faster evaluation cycles of RAG architectures. RAG architecture

In [38]:
for chunk in tqdm(semantic_chunks, desc="Ingesting to Neo4j"):
    triplets = extract_triplets(chunk)
    print(triplets)
    parsed = json.loads(triplets)
    triplets = parsed.get("triplets", [])
    save_to_neo4j(triplets)

print("\nKnowledge Graph constructed in Neo4j container.")

Ingesting to Neo4j:   0%|          | 0/18 [00:00<?, ?it/s]

{
  "triplets": [
       ["Ragas", "is", "framework"],
       ["Ragas", "provides", "reference-free evaluation"],
       ["RAG systems", "are composed of", "retrieval module"],
       ["RAG systems", "are composed of", "LLM based generation module"],
       ["RAG systems", "provide", "knowledge"],
       ["RAG systems", "reduce", "risk of hallucinations"],
       ["RAG architectures", "are challenging to evaluate", "because of several dimensions"],
       ["RAG architectures", "consider", "ability to identify relevant context passages"],
       ["RAG architectures", "consider", "ability to exploit passages"],
       ["RAG architectures", "consider", "quality of generation"],
       ["Ragas", "provides", "suite of metrics"],
       ["Ragas", "contributes to", "faster evaluation cycles"],
       ["RAG architectures", "are adopted quickly", "due to fast adoption of LLMs"],
       ["Language Models", "capture", "knowledge about the world"],
       ["Language Models", "answer", "questions"]

Ingesting to Neo4j:   6%|▌         | 1/18 [00:05<01:34,  5.55s/it]

{
  "triplets": [
       ["Ragas framework", "integrates with", "llama-index"],
       ["Ragas framework", "integrates with", "Langchain"],
       ["llama-index", "is used for", "building RAG solutions"],
       ["Langchain", "is used for", "building RAG solutions"],
       ["Ragas", "can be integrated into", "standard workflows"],
       ["LLMs", "are used for", "estimating faithfulness"],
       ["LLMs", "struggle with", "detecting hallucination"],
       ["standard prompting strategies", "are used for", "detecting hallucination"],
       ["external knowledge base", "is used for", "linking generated responses"],
       ["BARTScore", "estimates", "factuality"],
       ["BARTScore", "looks at", "conditional probabilities of generated text"],
       ["Kadavath et al.", "studied", "detecting hallucination"]
   ]
}


Ingesting to Neo4j:  11%|█         | 2/18 [00:06<00:44,  2.80s/it]

...

{
  "triplets": [
       ["Ragas", "supports", "development of quality assessment framework"],
       ["Ragas", "is", "framework"],
       ["Ragas", "provides", "feedback to developers of RAG systems"],
       ["Ragas", "is easy to use", "use"],
       ["GPT Ranking", "is", "baseline"],
       ["Broken ranking system", "is", "reference"],
       ["WikiEval", "is", "dataset"],
       ["WikiEval", "contains", "human judgements"],
       ["Evaluation framework", "should assess", "faithfulness"],
       ["Evaluation framework", "should assess", "answer relevance"],
       ["Evaluation framework", "should assess", "context relevance"],
       ["Ragas", "predicts", "faithfulness"],
       ["Ragas", "predicts", "answer relevance"],
       ["Ragas", "predicts", "context relevance"],
       ["Amos Azaria", "published", "The internal state of an LLM knows when its lying"],
       ["Tom M. Mitchell", "published", "The internal state of an LLM knows when its lying"],
       ["Sebastian Borgeaud", 

Ingesting to Neo4j:  94%|█████████▍| 17/18 [00:22<00:01,  1.88s/it]

{
  "triplets": [
       ["Robert Oppenheimer", "was", "theoretical physicist"],
       ["J. Robert Oppenheimer", "was pivotal in developing", "first nuclear weapons"],
       ["Manhattan Project", "ushered in", "Atomic Age"],
       ["Cillian Murphy", "stars as", "Oppenheimer"],
       ["Emily Blunt", "stars as", "Katherine 'Kitty' Oppenheimer"],
       ["Christopher Nolan", "directed", "film Oppenheimer"],
       ["PSLV-C56 mission", "is scheduled to be launched on", "Sunday, 30 July 2023"],
       ["PSLV-C56 mission", "will be launched from", "Satish Dhawan Space Centre"],
       ["Satish Dhawan Space Centre", "is located in", "Sriharikota, Andhra Pradesh, India"],
       ["Chimnabai Clock Tower", "was completed in", "1896"],
       ["Chimnabai Clock Tower", "is situated in", "Raopura area of Vadodara, Gujarat, India"],
       ["Chimnabai Clock Tower", "was named in memory of", "Chimnabai I"],
       ["Chimnabai I", "was", "queen"],
       ["Chimnabai I", "was first wife of", "Sayaj

Ingesting to Neo4j: 100%|██████████| 18/18 [00:26<00:00,  1.47s/it]


Knowledge Graph constructed in Neo4j container.
